In [1]:
# GET SAMPLE DATA

import requests
import json

# Download the data
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json"
)

# Load the data so you can see what it is
data = json.loads(resp.text)

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))


<class 'list'> 1000
{
  "Air Date": "2005-11-18",
  "Round": "Jeopardy!",
  "Value": 200,
  "Category": "RHYME TIME",
  "Question": "Any pigment on the wall so faded you can barely see it",
  "Answer": "faint paint"
}


In [6]:
# CONNECT

import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    port=8080,
    headers={
        "X-OpenAI-Api-Key": os.environ[
            "OPENAI_API_KEY"
        ]  # Replace with your OpenAI API key
    }
)

client.is_ready()



True

In [7]:
weaviate.__version__

'4.4b0'

In [91]:
# OLD CLIENT
# Connect

import weaviate
import os

old_client = weaviate.Client(
    url="http://localhost:8080/",  # Replace with your endpoint
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv(
            "OPENAI_API_KEY"
        )  # Replace with your OpenAI API key
    },
)

print(old_client.is_ready())

True


In [21]:
if (client.collections.exists("JeopardyCategory")):
    client.collections.delete("JeopardyCategory")

if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="title", data_type=wvc.DataType.TEXT),
    ]
)

In [23]:
# CrossRef Definition

client.collections.create(
    name="JeopardyQuestion",
    description="A Jeopardy! question",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="question", data_type=wvc.DataType.TEXT),
        wvc.Property(name="answer", data_type=wvc.DataType.TEXT),
        # highlight-start
        wvc.ReferenceProperty(
            name="hasCategory",
            target_collection="JeopardyCategory"
        )
        # highlight-end
    ]
)

client.collections.list_all()

{'Question': _CollectionConfigSimple(name='Question', description='A Jeopardy! question', generative_config=_GenerativeConfig(generator=<GenerativeSearches.OPENAI: 'generative-openai'>, model={}), properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='title', tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai'), _Property(data_type=<DataType.TEXT: 'text'>, description="This property was generated by Weaviate's auto-schema feature on Fri Nov 17 18:01:19 2023", index_filterable=True, index_searchable=True, name='answer', tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=False), vectorizer='text2vec-openai'), _Property(data_type=<DataType.TEXT: 'text'>, description="This property was generated by Weaviate's auto-schema feature on Fri Nov 17 18

In [25]:
import weaviate_datasets
dataset = weaviate_datasets.JeopardyQuestions1k()  # instantiate dataset

old_client = weaviate.Client("http://localhost:8080")
dataset = weaviate_datasets.JeopardyQuestions10k()  # instantiate dataset
dataset.upload_objects(old_client, 100)


/Users/davecuthbert/src/weaviate-io/.venv/lib/python3.10/site-packages/weaviate/warnings.py:131: DeprecationWarning: Dep006: You are using the `client.batch()` method, which will be removed in the next major release.
            Please instead use the `client.batch.configure()` method to configure your batch and `client.batch` to enter the context manager.
            See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(
10000it [00:27, 358.94it/s]


True

In [2]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyCategory")
response = jeopardy.query.fetch_objects(
    # Object IDs are included by default with the `v4` client! :)
    limit=1
)


# To print the results, uncomment this code.
for o in response.objects:
    print(o.uuid)

007e6d1f-6987-5728-a7cf-571cf02fccb8


In [27]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    # Object IDs are included by default with the `v4` client! :)
    limit=1
)


# To print the results, uncomment this code.
for o in response.objects:
    print(o.uuid)

0002bf92-80c8-5d94-af34-0d6c5fea1aaf


In [29]:
data_object = old_client.data_object.get_by_id(
    '00ff6900-e64f-5d94-90db-c8cfa3fc851b',
    class_name='JeopardyQuestion',
)

print(json.dumps(data_object, indent=2))



{
  "class": "JeopardyQuestion",
  "creationTimeUnix": 1701114330551,
  "id": "00ff6900-e64f-5d94-90db-c8cfa3fc851b",
  "lastUpdateTimeUnix": 1701114331755,
  "properties": {
    "air_date": "2008-11-10T00:00:00Z",
    "answer": "San Francisco",
    "hasCategory": [
      {
        "beacon": "weaviate://localhost/JeopardyCategory/fec50326-dfa1-53c9-90e8-63d0240bd933",
        "href": "/v1/objects/JeopardyCategory/fec50326-dfa1-53c9-90e8-63d0240bd933"
      }
    ],
    "points": 400,
    "question": "This city's Cable Car Museum has one from fhe first company, the Clay Street Hill Railroad",
    "round": "Double Jeopardy!"
  },
  "vectorWeights": null
}


In [30]:
data_object = old_client.data_object.get_by_id(
    '20ffc68d-986b-5e71-a680-228dba18d7ef',
    class_name='JeopardyCategory',
)

print(json.dumps(data_object, indent=2))

{
  "class": "JeopardyCategory",
  "creationTimeUnix": 1701114339856,
  "id": "20ffc68d-986b-5e71-a680-228dba18d7ef",
  "lastUpdateTimeUnix": 1701114339856,
  "properties": {
    "title": "U.S. CITIES"
  },
  "vectorWeights": null
}


In [3]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=3,
    return_properties=["question", "answer", "points"]
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "points": 100.0,
  "answer": "Jonah",
  "question": "This prophet passed the time he spent inside a fish offering up prayers"
}
{
  "points": 400.0,
  "answer": "lay eggs",
  "question": "Pythons are oviparous, meaning they do this"
}
{
  "points": 800.0,
  "answer": "Wal-Mart",
  "question": "In August 1996 this chain, the world's largest retailer, opened its first outlets in China"
}


In [4]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=1
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{}


In [37]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=[
        wvc.FromReference(
            link_on="hasCategory",
            return_properties=["title"]
        ),
        "question",
    ],
    limit=2
)

# print result objects
for o in response.objects:
    print(o.properties["question"])
    # print referenced objects
    for ref in o.properties["hasCategory"].objects:
        print(ref.properties)

This prophet passed the time he spent inside a fish offering up prayers
{'title': 'THE BIBLE'}
Pythons are oviparous, meaning they do this
{'title': 'ANIMALS'}


In [11]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    return_properties=["question", "answer"],
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Famine",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}


In [7]:
response = (
    old_client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
        query="food"
    )
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "JeopardyQuestion": [
        {
          "answer": "a closer grocer",
          "question": "A nearer food merchant"
        },
        {
          "answer": "food stores (supermarkets)",
          "question": "This type of retail store sells more shampoo & makeup than any other"
        },
        {
          "answer": "Famine",
          "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
        }
      ]
    }
  }
}


In [14]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
)
    

# This prints the response object
print(response)

# This formatting step prints the output that you probably want.
# The remaining examples use formatted output where appropriate.
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

_QueryReturn(objects=[_Object(uuid=UUID('0002bf92-80c8-5d94-af34-0d6c5fea1aaf'), metadata=None, properties={'answer': 'Jonah', 'question': 'This prophet passed the time he spent inside a fish offering up prayers'}, vector=None), _Object(uuid=UUID('00031ce9-ef72-5447-a02a-851f221d7359'), metadata=None, properties={'answer': 'lay eggs', 'question': 'Pythons are oviparous, meaning they do this'}, vector=None), _Object(uuid=UUID('0003b234-14d3-5ea6-b4b5-0fb0a6dcc43c'), metadata=None, properties={'answer': 'Wal-Mart', 'question': "In August 1996 this chain, the world's largest retailer, opened its first outlets in China"}, vector=None), _Object(uuid=UUID('00059a05-3e5a-5eba-97b6-3eadba5f8682'), metadata=None, properties={'answer': 'the Hubble telescope', 'question': 'In 1990 this space telescope provided the first picture of Charon as completely separate from Pluto'}, vector=None), _Object(uuid=UUID('000b7966-a62a-5a0e-bac4-10393dad0d23'), metadata=None, properties={'answer': 'Hank Aaron', 

In [15]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    limit=1
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "Jonah",
  "question": "This prophet passed the time he spent inside a fish offering up prayers"
}


In [18]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    limit=1,
    offset=1
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "lay eggs",
  "question": "Pythons are oviparous, meaning they do this"
}


In [19]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    include_vector=True,
    limit=1
)

print(response.objects[0].vector)

[0.006506513338536024, -0.017786195501685143, 0.005879146046936512, 0.006707012187689543, -0.019209090620279312, 0.006060241721570492, -0.00979857612401247, -0.009119466878473759, -0.01574239879846573, -0.01975237764418125, 0.026983274146914482, 0.03546890616416931, 0.022714586928486824, -0.009048322215676308, 0.009397578425705433, 0.008841355331242085, 0.02580615133047104, 0.017191166058182716, 0.03246789053082466, -0.013213525526225567, 0.00848563201725483, -0.009138870052993298, -0.0012345236027613282, 0.001901505864225328, 0.015988172963261604, 0.0033211675472557545, 0.025482766330242157, -0.019842926412820816, 0.004575903061777353, -0.01394437626004219, 0.009953801520168781, -0.0012636282481253147, -0.0257544107735157, -0.02046382613480091, -0.019364316016435623, -0.014099601656198502, -0.013517508283257484, 0.004317194689065218, 0.021136466413736343, 0.018420029431581497, 0.014306568540632725, -0.015949366614222527, -0.005090085789561272, 0.011984661221504211, -0.0341753661632537

In [20]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    # Object IDs are included by default with the `v4` client! :)
    limit=1
)

for o in response.objects:
    print(o.uuid)
    

0002bf92-80c8-5d94-af34-0d6c5fea1aaf


In [21]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=[
        wvc.FromReference(
            link_on="hasCategory",
            return_properties=["title"]
        ),
        "question",
    ],
    limit=2
)

# print result objects
for o in response.objects:
    print(o.properties["question"])
    # print referenced objects
    for ref in o.properties["hasCategory"].objects:
        print(ref.properties)

This prophet passed the time he spent inside a fish offering up prayers
{'title': 'THE BIBLE'}
Pythons are oviparous, meaning they do this
{'title': 'ANIMALS'}


In [24]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    return_properties=["question", "answer"],
    query="food",
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Famine",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}


In [26]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    alpha=0.5,
    return_properties=["question", "answer"],
    return_metadata=wvc.MetadataQuery(score=True),
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    print(o.metadata.explain_score, o.metadata.score)

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
None 0.016133230179548264
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
None 0.01577247865498066
{
  "answer": "Famine",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}
None 0.014474772848188877


In [27]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
   query="food",
   return_properties=["question", "answer"],
   limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Famine",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}


In [28]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
   query="food",
   return_properties=["question", "answer"],
   auto_limit=1
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "honey",
  "question": "The primary source of this food is the Apis mellifera"
}


In [30]:
client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    return_properties=["question", "answer"],
    query="food", 
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "Smelt",
  "question": "A silvery food fish, or to heat ore in order to extract metal"
}


In [33]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
  query="food",
  return_properties=["question", "answer"],
  fusion_type=wvc.HybridFusion.RELATIVE_SCORE,
  limit=3
)
 
#print(response)
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Famine",
  "question": "From the Latin for \"hunger\", it's a period when food is extremely scarce"
}


In [35]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    query_properties=["question"],
    return_properties=["question", "answer"],
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "Smelt",
  "question": "A silvery food fish, or to heat ore in order to extract metal"
}
{
  "answer": "pasta",
  "question": "Type of food that comes in shapes of bow ties, elbows & wagon wheels"
}


In [37]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    query_properties=["question^2", "answer"],
    return_properties=["question", "answer"],
    alpha=0.25,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Smelt",
  "question": "A silvery food fish, or to heat ore in order to extract metal"
}


In [42]:
query_vector = [0.013085687533020973, -0.00777443777769804, 0.005439540836960077, -0.021052561700344086, -0.02164270170032978, -0.006985447835177183, -0.018974246457219124, -0.025260508060455322, -0.0013630924513563514, -0.03597281128168106, 0.027993107214570045, 0.01635710895061493, -0.02948128432035446, 0.009159981273114681, -0.0026780758053064346, 0.001033544773235917, 0.04051431640982628, 0.001919554895721376, 0.024952609091997147, -0.01960287243127823, 0.0001997531799133867, 0.031405650079250336, 0.021142365410923958, -0.007954045198857784, -0.0008338918560184538, -0.0040572043508291245, 7.381747127510607e-05, -0.019051222130656242, 0.004942412953823805, -0.01888444274663925, 0.028121398761868477, 0.004631306976079941, -0.031559597700834274, -0.003143130801618099, 0.01867917738854885, -0.024208521470427513, 0.0056351847015321255, -0.019333461299538612, -0.0012756941141560674, 0.01737060956656933, 0.020128866657614708, -0.007755194325000048, -0.0071329823695123196, -0.007761608809232712, -0.0074986121617257595, 0.00579554820433259, 0.002782312221825123, 0.01349621918052435, 0.003954571671783924, 0.003170392708852887, 0.017678506672382355, 0.008980373851954937, -0.027454284951090813, -0.004377932287752628, 0.013547535054385662, 0.028506271541118622, -0.011225467547774315, -0.003855146234855056, -8.654634439153597e-05, -0.021770991384983063, -0.004724318161606789, 0.037281379103660583, -0.0543954074382782, 0.01912819594144821, 0.009916898794472218, -0.007806510664522648, 0.0035921495873481035, 0.011757874861359596, -0.004980900324881077, -0.014381427317857742, 0.005952704697847366, 0.009839924052357674, -0.02256639674305916, 0.014561034739017487, 0.01888444274663925, 0.0006859562126919627, 0.0024984683841466904, 0.0033355674240738153, -0.007633317727595568, -0.015330781228840351, 0.02297692745923996, -0.02996879070997238, 0.017473241314291954, 0.01232235599309206, 0.019949259236454964, 0.009769363328814507, -0.038307707756757736, 0.0278134997934103, 0.012264625169336796, -0.007062422577291727, -0.013316611759364605, -0.00465055089443922, 0.013188320212066174, 0.008210627362132072, -0.023862136527895927, 0.006998276803642511, -0.005157300271093845, 0.036024127155542374, -0.0211167074739933, -0.0013109742430970073, -0.0014577071415260434, 0.00858267117291689, -0.004637721460312605, -0.0176656786352396, -0.0035793203860521317, -0.004618478007614613, -0.0015691599110141397, -0.018692007288336754, 0.016536716371774673, -0.012476304545998573, -0.03150828182697296, 0.029789183288812637, 0.01989794336259365, -0.03304777666926384, -0.025234850123524666, 0.0034093346912413836, 0.04154064506292343, 0.013188320212066174, 4.1249832065659575e-06, -0.015279464423656464, -0.01488176267594099, 0.016498230397701263, 0.03025102987885475, -0.015574533492326736, 0.033304356038570404, 0.014445573091506958, 0.018114697188138962, -0.011077932082116604, -0.020719004794955254, -0.008069506846368313, 0.0259789377450943, 0.013079272583127022, 0.014920249581336975, -0.005500479135662317, -0.025119388476014137, 0.017062710598111153, -0.0018938967259600759, -0.003059741575270891, -0.005179751198738813, -0.0017062709666788578, 0.02025715820491314, 0.039077453315258026, -0.01520248968154192, 0.012912494130432606, -0.018730493262410164, 0.022669028490781784, 0.014458402059972286, 0.01916668377816677, -0.013406415469944477, 0.0027758977375924587, 0.02151441015303135, -0.025080900639295578, 0.017999235540628433, -0.01716534234583378, 0.014445573091506958, 0.014868932776153088, -0.008326089009642601, 0.021540068089962006, -0.009820680133998394, 0.0033516038674861193, -0.006863571237772703, 0.0016012326814234257, 0.00046625779941678047, -0.014419914223253727, 0.009442221373319626, 0.021976256743073463, 0.01558736339211464, 0.005356151610612869, 0.013104931451380253, -0.0007625299622304738, -0.0013294160598888993, 0.024580566212534904, -0.011308856308460236, 0.03032800555229187, -0.006594160106033087, 0.023041073232889175, -0.0036466731689870358, 0.004679416306316853, -0.01162958424538374, 0.031764864921569824, -0.01007084734737873, -0.004990521818399429, 0.049135472625494, 0.016062039881944656, -0.005660842638462782, -0.019217999652028084, 0.0008852082537487149, 0.011879751458764076, -0.00190031121019274, -0.03250895440578461, 0.006546051241457462, 0.04238736256957054, 0.014009383507072926, -0.012232552282512188, -0.6835347414016724, -0.006443418096750975, 0.003932120744138956, -0.02776218391954899, 0.027839159592986107, 0.035998471081256866, 0.0012444232124835253, 0.0010808521183207631, 0.005522930063307285, -0.01676764152944088, -0.012976639904081821, 0.010468550026416779, -0.003197654616087675, 0.007274102885276079, -0.00998745858669281, -0.025119388476014137, 0.030302347615361214, -0.01453537680208683, -0.04323408380150795, 0.023002585396170616, -0.02207889035344124, 0.02712072804570198, -0.007633317727595568, -0.0038294880650937557, 0.025991767644882202, -0.002905792323872447, 0.004855816252529621, -0.019949259236454964, 0.00048149237409234047, -0.006250981707125902, -0.017691336572170258, -0.005445955321192741, 0.0037910006940364838, -0.01242498867213726, 0.0669679269194603, -0.00747936824336648, -0.015574533492326736, 0.027505602687597275, 0.024336813017725945, 0.0389748215675354, -0.006587745621800423, -0.012655912898480892, 0.022014744579792023, 0.017524557188153267, -0.009442221373319626, -0.015536046586930752, 0.038102444261312485, 0.016613692045211792, 0.021873624995350838, -0.013098516501486301, 0.025080900639295578, 0.019102538004517555, -0.01688310317695141, 0.014137674123048782, 0.012752130627632141, -0.004724318161606789, 0.031610917299985886, -0.024349642917513847, 0.02603025548160076, 0.010269698686897755, -0.007941216230392456, 0.00834533292800188, 0.0005380206275731325, -0.03150828182697296, 0.0002830421435646713, -0.021629871800541878, -0.012681570835411549, 0.01114207785576582, 0.030995119363069534, 0.0018008856568485498, -0.0017800383502617478, 0.021950598806142807, -0.003095021704211831, -0.00533690769225359, 0.018935760483145714, 0.015279464423656464, 0.019384779036045074, 0.009589755907654762, 0.005211824085563421, 0.0006318334490060806, -0.010859837755560875, 0.00035219904384575784, -0.030481955036520958, 0.0026139302644878626, 0.006504356395453215, 0.008229871280491352, -0.03661426529288292, 0.005215031560510397, 0.0012764959828928113, 0.013175491243600845, 0.02907075360417366, 0.0025481809861958027, -0.004807707387953997, -0.012739301659166813, 0.0014464816777035594, -0.0017463619587942958, -0.03435634449124336, 0.004156630020588636, -0.0032634036615490913, -0.016818957403302193, -0.018127525225281715, 0.015061370097100735, 0.00990406982600689, -0.0008860100642777979, 0.01276495959609747, 0.0259789377450943, 0.01045572105795145, 0.034407660365104675, 0.02747994288802147, -0.042566969990730286, 0.014407085254788399, -0.00013610879250336438, -0.013970895670354366, -0.01984662562608719, 0.003755720565095544, -0.020680518820881844, 0.01708836853504181, -0.010648157447576523, 0.005221446044743061, -0.035177405923604965, -0.0045286742970347404, -0.01708836853504181, 0.013034370727837086, -0.0012845142045989633, -0.0063087125308811665, 0.03338133171200752, 0.012290283106267452, -0.01463800948113203, -0.021296314895153046, -0.010789277963340282, 0.015766970813274384, 0.010949641466140747, 0.011308856308460236, -0.019833797588944435, 0.012848349288105965, 0.007152226287871599, 0.020205840468406677, -0.016331451013684273, 0.013855434022843838, -0.01640842668712139, -0.008107994683086872, -0.010872666724026203, -0.007242029998451471, -0.006908473093062639, 0.004240019246935844, -0.025119388476014137, 0.00765256118029356, 0.0022595261689275503, -0.010179894976317883, -0.009474294260144234, 0.002902585081756115, -0.00389684084802866, -0.021873624995350838, -0.0032698181457817554, 0.023130876943469048, -0.019577214494347572, -0.004086070228368044, -0.006052130367606878, 0.009134323336184025, -0.015125514939427376, -0.0040604118257761, 0.020808808505535126, -0.010808520950376987, 0.016652178019285202, -0.021206511184573174, -0.014625180512666702, 0.002657228382304311, -0.002690904773771763, 0.007165055256336927, -0.03312474861741066, 0.024208521470427513, -0.013060029596090317, 0.010077262297272682, 0.019012734293937683, -0.014304452575743198, 0.033791862428188324, -0.02046242356300354, 0.009673145599663258, 0.019679848104715347, 0.004188702907413244, 0.0067865969613194466, -0.028480613604187965, -0.029686549678444862, -0.0015900072176009417, 0.013791288249194622, -0.0070880805142223835, 0.03576754406094551, 0.022964099422097206, -0.026402298361063004, 0.02801876701414585, -0.022348301485180855, -0.000972606532741338, 0.0023493298795074224, 0.0028400432784110308, 0.004807707387953997, -0.0026235519908368587, 0.02513221837580204, 0.02727467752993107, 0.007575586903840303, -0.018178842961788177, 0.015818286687135696, -0.021142365410923958, 0.018538057804107666, -0.02119368128478527, 0.027428627014160156, -0.016626520082354546, -0.004506223369389772, -0.014445573091506958, 0.018704835325479507, 0.020411105826497078, 0.035536620765924454, -0.026761513203382492, -0.032072763890028, 0.005173336714506149, 0.004531881306320429, 0.036793872714042664, 3.102037589997053e-05, 0.0017367401160299778, 0.006250981707125902, 5.0188864406663924e-05, 0.023733844980597496, -0.015497559681534767, -0.0013462542556226254, -0.008255529217422009, -0.039077453315258026, 0.022438105195760727, -0.013380756601691246, 0.010487793944776058, 0.002185758901759982, -0.026864146813750267, -0.0071714697405695915, 0.02582498826086521, -0.006501149386167526, 0.0024663954973220825, 0.02524768002331257, -0.0340997613966465, -0.005564624443650246, -0.022835807874798775, 0.008512111380696297, 0.005567831918597221, -0.007004691753536463, 0.028711538761854172, 0.022104548290371895, 0.014253135770559311, 0.012604596093297005, -0.01599789410829544, 0.027454284951090813, 0.003919291775673628, -0.026735855266451836, 0.00616117799654603, 0.0073767355643212795, 0.0033099092543125153, -0.027556918561458588, 0.006097032222896814, 0.020244328305125237, 0.006318334490060806, 0.008993202820420265, -0.0174604132771492, 0.014676496386528015, 0.017075538635253906, 0.017203830182552338, 0.007190713658928871, 0.015895262360572815, 0.0021151986438781023, 0.005086740478873253, -0.0014793562004342675, -0.013675826601684093, -0.024939781054854393, -0.019949259236454964, -0.0022739588748663664, 0.0009325155988335609, -0.026043083518743515, 0.00585648650303483, 0.007030349690467119, -0.0026059120427817106, -0.010385161265730858, -0.015689995139837265, 0.01193748228251934, -0.0043073720298707485, -0.01148846372961998, -0.033920153975486755, -0.0259789377450943, 0.02991747297346592, 0.011841264553368092, -0.006587745621800423, 0.0064370036125183105, -0.01544624287635088, 0.022361131384968758, -0.011090761981904507, -0.008403063751757145, 0.02554274909198284, 0.014445573091506958, 0.0013510651187971234, 0.010301771573722363, -0.0035504549741744995, 0.026761513203382492, 0.003954571671783924, -0.02329765446484089, 0.014137674123048782, 0.005426711402833462, 0.012245381250977516, -0.025966109707951546, -0.01708836853504181, -0.008871326223015785, 0.03145696595311165, 0.009756534360349178, -0.007011106237769127, 0.00013781266170553863, -0.00096539017977193, 0.005487649701535702, 0.003014839719980955, 0.005817999131977558, -0.0025032791309058666, 0.004454907029867172, 0.030225371941924095, -0.012816276401281357, -0.015330781228840351, -0.004721110686659813, 0.009102250449359417, 0.0011610339861363173, -0.013714313507080078, -0.05208616703748703, -0.04315711185336113, 0.01729363389313221, 0.05216314271092415, 0.01325246598571539, 0.016947248950600624, 0.008524940349161625, 0.0024663954973220825, -0.024182863533496857, -0.03697348013520241, 0.003208880079910159, 8.890168828656897e-05, -0.021129537373781204, 0.012534036301076412, -0.009121494367718697, 0.01232235599309206, -0.005317664239555597, 0.012976639904081821, -0.019949259236454964, 0.012957395985722542, -0.017588702961802483, 0.027916133403778076, -0.019102538004517555, 0.014920249581336975, 0.012636668980121613, 0.02127065695822239, 0.019551556557416916, 0.0340997613966465, 0.02363121137022972, 0.04351632669568062, 0.025504261255264282, 0.010000287555158138, -0.02642795629799366, -0.011828435584902763, 0.006218908820301294, -0.0006951771210879087, 0.0045126378536224365, -0.02034696191549301, -0.006462662015110254, 0.02955825813114643, 0.006122690625488758, 0.01676764152944088, -0.011828435584902763, 0.005975155625492334, 0.021052561700344086, -0.009467880241572857, -0.01712685637176037, 0.017896601930260658, -0.026966780424118042, -0.017139684408903122, 0.04136103764176369, -0.01558736339211464, -0.023887794464826584, 0.0255812369287014, 0.0028288178145885468, 0.006857156753540039, -0.03340699151158333, 0.021989086642861366, 0.009224127046763897, 0.009467880241572857, -0.0012612614082172513, -0.013027956709265709, -0.017601532861590385, -0.00846720952540636, -0.02196342870593071, -0.008332503959536552, -0.009089421480894089, -0.01669066585600376, -0.013342269696295261, -0.0176528487354517, -0.011475634761154652, -0.022540738806128502, 0.010173480957746506, 0.010847008787095547, -0.012829105369746685, 0.007395979482680559, -0.0013582815881818533, 0.01985945552587509, 0.010616084560751915, -0.0229384396225214, -0.008396649733185768, 0.020116036757826805, 0.022964099422097206, -0.016793299466371536, -0.015600192360579967, 0.002857683226466179, -0.00562235526740551, -0.010096506215631962, 0.010442892089486122, 0.009384490549564362, -0.035741887986660004, 0.020372619852423668, 0.0012748923618346453, -0.016536716371774673, 0.024144375696778297, 0.0035312112886458635, -0.006420967169106007, -0.006241359747946262, -0.007274102885276079, -0.005288798827677965, -0.0012805050937458873, -0.01652388833463192, -0.03604978695511818, -0.014830445870757103, 0.015317952260375023, -0.01439425628632307, -0.007062422577291727, -0.003717233194038272, 0.01530512236058712, 0.0011818812927231193, -0.011007372289896011, 0.0003121080808341503, 0.007594830356538296, 0.019577214494347572, -0.01007084734737873, 0.005054667592048645, 0.022155864164233208, -0.02464471198618412, 0.019782479852437973, -0.008415892720222473, 0.023810818791389465, -0.0044516995549201965, -0.013483390212059021, 0.0035344185307621956, -0.02383647859096527, 0.008184969425201416, -0.0033580183517187834, 0.012950981967151165, -0.0018522021127864718, 0.0011353758163750172, 0.0035023458767682314, -0.006869985722005367, -0.010686644352972507, -0.010288942605257034, 0.02209172025322914, -0.0031655817292630672, -0.02996879070997238, -0.008184969425201416, -0.016049209982156754, 0.01286759227514267, 0.006882815156131983, -0.01534361019730568, -0.02663322351872921, -0.0021617042366415262, -0.008178554475307465, 3.533015478751622e-05, 0.0031046434305608273, -0.0066198185086250305, -0.02991747297346592, 0.0032185018062591553, -0.0324832946062088, 0.004830158315598965, 0.03543398901820183, -0.006206079851835966, -0.014817616902291775, -0.016049209982156754, 0.006026472430676222, -0.016331451013684273, -0.03153394162654877, -0.026607563719153404, -0.015882432460784912, -0.005872522946447134, 0.042361706495285034, 0.023246338590979576, 0.0025674246717244387, 0.023438775911927223, -0.013829775154590607, -0.022874295711517334, 0.006241359747946262, -0.004833365324884653, 0.005878937430679798, -0.013662997633218765, 0.018845954909920692, 0.045645955950021744, 0.015933748334646225, -0.018050551414489746, -0.0034510293044149876, -0.010115749202668667, 0.031200384721159935, -0.009166396223008633, -0.022296985611319542, -0.01138583105057478, -0.022399617359042168, 0.022258497774600983, 0.00609061773866415, -0.02034696191549301, 0.025029584765434265, 0.04243868216872215, -0.013290952891111374, 0.01660086214542389, -0.009269028902053833, 0.018499569967389107, 0.016780469566583633, 0.009467880241572857, -0.023477263748645782, 0.01216199155896902, -0.022887123748660088, -0.0064754909835755825, -0.014868932776153088, -0.012918909080326557, 0.01221972331404686, -0.02399042621254921, -0.0015306726563721895, 0.008993202820420265, 0.009262613952159882, -0.006145141553133726, -0.02159138396382332, -0.01359885185956955, 0.010808520950376987, -0.0158311165869236, 0.0031944471411406994, 0.005423504393547773, -0.010641742497682571, -0.019230829551815987, -0.01965419016778469, -0.020808808505535126, -0.022553566843271255, -0.00031290989136323333, 0.008685304783284664, -0.03953930363059044, 0.02529899589717388, -0.02062920108437538, -0.027736525982618332, 0.007447295822203159, -0.0034991384018212557, 0.02927601896226406, -0.002690904773771763, 0.022630542516708374, -0.0075820013880729675, 0.010628913529217243, -0.03130301833152771, 0.0015370871406048536, -0.0008515318622812629, 0.004493393935263157, 0.0011746649397537112, 0.006767353042960167, 0.030789852142333984, -0.022669028490781784, 0.007966874167323112, 0.017922259867191315, -0.012258210219442844, -0.028121398761868477, 0.020616373047232628, 0.0074152229353785515, 0.017486071214079857, -0.026530589908361435, -0.022887123748660088, -0.015779798850417137, 0.005946290213614702, -0.010134993121027946, 0.015600192360579967, -0.008370990864932537, -0.003912877291440964, -0.01920517161488533, 0.030174056068062782, -0.02866022102534771, 0.016960076987743378, -0.0009373265202157199, -0.011693730019032955, 0.0024744137190282345, -0.03438200429081917, -0.036845192313194275, 0.013778459280729294, 0.010423648171126842, 0.03720440715551376, -0.009435807354748249, 0.018730493262410164, 0.0051861656829714775, 0.014663667418062687, -0.014779129065573215, -0.003261800156906247, 0.012931738048791885, 0.020321302115917206, -0.02894246205687523, -0.03982154279947281, -0.013027956709265709, -0.012348013930022717, 0.008787937462329865, -0.03263724595308304, -0.016434084624052048, 0.023605553433299065, -0.00012598582543432713, -0.01669066585600376, 0.00453829625621438, 0.006190043408423662, 0.004076448269188404, 0.009346003644168377, 0.004714696202427149, -0.009269028902053833, -0.028121398761868477, 0.00096539017977193, 0.006196457892656326, -0.012091431766748428, -0.017062710598111153, -0.028403639793395996, 0.0158311165869236, -0.00407965574413538, 0.012213308364152908, -0.013650167733430862, 8.679691381985322e-05, 0.00011275580618530512, -0.021706845611333847, 0.02098841592669487, -0.03250895440578461, 0.013509048148989677, 0.004486979451030493, 0.006818669382482767, -0.015189660713076591, 0.004679416306316853, 0.018050551414489746, 0.003131905337795615, -0.005478028208017349, -0.018807468935847282, 0.0045511252246797085, 0.017383437603712082, 0.01033384446054697, -0.0013133796164765954, 0.03284250944852829, 0.007120153401046991, -0.0002808371209539473, -0.0225279089063406, -0.018910100683569908, 0.022681858390569687, -0.007857827469706535, -0.0041854954324662685, -0.008326089009642601, -0.023900622501969337, 0.016472570598125458, 0.01903839223086834, -0.0012067377101629972, 0.005118812900036573, -0.010891910642385483, 0.01415050309151411, -0.016536716371774673, 0.020885784178972244, -0.000979822943918407, -0.00815931148827076, -0.005192580632865429, 0.010693059302866459, -0.015433413907885551, 0.011905409395694733, -0.01737060956656933, -0.0032521781977266073, 0.0033259454648941755, 0.058603353798389435, 0.004692245274782181, 0.008236285299062729, 0.0031463380437344313, -0.019833797588944435, -0.02301541529595852, -0.022309813648462296, -0.0017174964305013418, -0.02359272539615631, -0.010077262297272682, 0.01162958424538374, 0.009878410957753658, -0.013868262991309166, -0.02354140765964985, 0.03019971400499344, -0.015972236171364784, -0.0067032077349722385, -0.015215318650007248, 0.004316993989050388, 0.02472168579697609, 0.006780182011425495, -0.03345830738544464, 0.012277454137802124, 0.01171297300606966, 0.011180565692484379, 0.008447965607047081, 0.004608856048434973, 0.014137674123048782, -0.0196926761418581, 0.018858784809708595, -0.002015773206949234, -0.027659550309181213, -0.02025715820491314, 0.008499282412230968, 0.007973289117217064, 0.025837818160653114, 0.0005063487333245575, -0.013534706085920334, -0.02408022992312908, -0.044722262769937515, -0.01232235599309206, 0.012617425061762333, 0.005747439339756966, 0.0075820013880729675, -0.017396267503499985, -0.014830445870757103, 0.0022771661169826984, 0.00034357947879470885, -0.028249690309166908, -0.005426711402833462, 0.008909814059734344, 0.007049593608826399, 0.021052561700344086, -0.0023284826893359423, -0.0052920058369636536, 0.00941656343638897, -0.026094401255249977, 0.00616117799654603, 0.003858353476971388, -0.01684461534023285, 0.008961129933595657, -0.009673145599663258, -0.011963141150772572, -0.014009383507072926, -0.019153853878378868, -0.01697290688753128, -0.007594830356538296, 0.02712072804570198, -0.00564159918576479, -0.00581479212269187, -0.021527238190174103, 0.006728865671902895, -0.0043073720298707485, 0.020552227273583412, -0.003710818709805608, 0.012918909080326557, 0.015882432460784912, 0.005898181349039078, 0.014432743191719055, -0.019461752846837044, 0.00024976665736176074, 0.0008150491048581898, 0.0021264241077005863, -0.008306846022605896, -0.005381809547543526, 0.010410819202661514, -0.010173480957746506, -0.002830421319231391, 0.019436094909906387, -0.017563045024871826, -0.020770322531461716, 0.004037960898131132, -0.018730493262410164, -0.00880718044936657, 0.011745045892894268, 0.18124960362911224, 0.010731546208262444, -0.0013294160598888993, 0.03794849291443825, 0.010956056416034698, 0.03723006322979927, 0.02935299277305603, 0.005340115167200565, 0.01033384446054697, 0.030558928847312927, -0.001111321267671883, 0.016754811629652977, -0.010641742497682571, 0.0006975826108828187, 0.00545878428965807, -0.01688310317695141, -0.014868932776153088, -0.025183534249663353, -0.009166396223008633, -0.013534706085920334, -0.010507036931812763, -0.019820967689156532, -0.013444902375340462, -0.0038294880650937557, 0.029583917930722237, 0.0178581140935421, -0.017973575741052628, -0.01335509866476059, 0.034895166754722595, 0.019346291199326515, 0.008088750764727592, -0.00846079457551241, 0.0025722356513142586, -0.00622211629524827, -0.02042393572628498, 0.014843274839222431, -0.005320871248841286, -0.018974246457219124, 0.009711632505059242, -0.012822690419852734, -0.007197128143161535, -0.00787707045674324, 0.008358161896467209, -0.03320172429084778, -0.020475251600146294, 0.0036113932728767395, -0.006940545979887247, 0.0025129010900855064, 0.005933461245149374, 0.008499282412230968, -0.019051222130656242, -0.026684539392590523, 0.0005632779211737216, 0.0278134997934103, 0.0119759701192379, 0.00838382076472044, 0.020924270153045654, 0.008916228078305721, 0.0006194052402861416, 0.005465198773890734, 0.003152752760797739, 0.03515174984931946, -0.013239637017250061, 0.029455626383423805, -0.017357779666781425, 0.016472570598125458, -0.01102020125836134, -0.019820967689156532, -0.0022017951123416424, -0.01359885185956955, 0.035741887986660004, -0.022476593032479286, -0.010705888271331787, -0.011110004968941212, -0.009179225191473961, -0.01716534234583378, 0.00860833004117012, 0.02606874145567417, -0.0032970800530165434, 0.019872283563017845, -0.014714984223246574, 0.007575586903840303, -0.01680612750351429, -0.0025658211670815945, -0.01439425628632307, -0.0373583547770977, 0.014843274839222431, -0.007421637419611216, -0.01692158915102482, -0.02561972290277481, -0.0077038779854774475, -0.0021151986438781023, -0.008281187154352665, 0.012809861451387405, 0.01684461534023285, -0.005240689497441053, -0.03835902363061905, 0.013765630312263966, 0.014265964739024639, -0.008075921796262264, -0.01684461534023285, -0.03864126652479172, 0.016010724008083344, 0.0005817197379656136, -0.025324653834104538, -0.016062039881944656, -0.013996553607285023, 0.012232552282512188, 0.008967544883489609, -0.007344662677496672, 0.014342939481139183, 0.004150215536355972, -0.004124557599425316, -0.005811584647744894, -0.006940545979887247, -0.008800766430795193, 0.008653231896460056, -0.010276113636791706, 0.021976256743073463, -0.012745716609060764, 0.03807678446173668, -0.003932120744138956, 0.0017559838015586138, 0.020885784178972244, -0.04903284087777138, -0.02650493197143078, -0.02160421386361122, 0.004836572799831629, 0.015420584939420223, -0.03271421790122986, 0.014330110512673855, -0.024696027860045433, -0.019872283563017845, 0.0034510293044149876, -0.006882815156131983, 0.000971804722212255, 0.006084203254431486, 0.01888444274663925, -0.029378650709986687, 0.001085663097910583, -0.007081666029989719, -0.0034927239175885916, 0.014253135770559311, -0.014330110512673855, -0.004567161668092012, -0.018409766256809235, 8.809987048152834e-05, -0.0016373145626857877, -0.0082234563305974, -0.010045189410448074, -0.0034959311597049236, -0.020359789952635765, -0.010276113636791706, -0.005295213311910629, 0.016087697818875313, -0.040411680936813354, -0.03256027027964592, -0.004791670944541693, -0.02062920108437538, 0.005747439339756966, -0.014009383507072926, -0.014060699380934238, 0.024991096928715706, -0.006174006965011358, -0.007434466388076544, -0.022515080869197845, -0.16339148581027985, 0.029609575867652893, 0.006722451187670231, -0.037332694977521896, 0.021976256743073463, -0.022361131384968758, -0.0043458594009280205, -0.005083533003926277, 0.009403734467923641, 0.0114692198112607, 0.020475251600146294, -0.01242498867213726, -0.04600517079234123, -0.031610917299985886, 0.02942996844649315, -0.007530685048550367, -0.0010920775821432471, 0.019667018204927444, 0.007312590256333351, 0.022194352000951767, 0.013983724638819695, -0.006613404024392366, 0.018358450382947922, -0.01757587492465973, -0.0015042126178741455, 0.021334802731871605, 0.004499808885157108, 0.01993642933666706, -0.014368598349392414, 0.008563428185880184, 0.012418573722243309, 0.0027261849027127028, 0.014766300097107887, 0.03820507600903511, 0.0012227741535753012, -0.01509985700249672, -0.01033384446054697, -0.0005817197379656136, 0.0033291529398411512, 0.016023552045226097, 0.010147822089493275, 0.027736525982618332, -0.002987577812746167, -0.0002890557807404548, -0.01903839223086834, 0.007235615514218807, 0.02907075360417366, -0.006546051241457462, 0.011000958271324635, -0.0002021586406044662, 0.0162416473031044, -0.013393585570156574, 0.0011882958933711052, 0.008043848909437656, 0.020282816141843796, 0.012328770011663437, -0.0011650430969893932, -0.0016252873465418816, 0.005708951968699694, 0.005198995117098093, -0.003941742703318596, 0.0032024653628468513, 0.014458402059972286, -0.00858267117291689, -0.00953844003379345, -0.02184796705842018, -0.02297692745923996, 0.010122164152562618, 0.0176656786352396, 0.005939875729382038, -0.004679416306316853, -0.002729392144829035, 0.0031607707496732473, -0.010494207963347435, 0.009461465291678905, 0.009929727762937546, -0.0269924383610487, -0.00016016335575841367, 0.009711632505059242, -0.01721666008234024, -0.0063087125308811665, -0.008204213343560696, -0.0033355674240738153, 0.012687984853982925, 0.004515845328569412, 0.005776304751634598, -0.009666730649769306, 0.0033387746661901474, -0.007780852261930704, -0.010115749202668667, 0.028506271541118622, -0.0077038779854774475, -0.007492197677493095, -0.02209172025322914, -0.011360173113644123, 0.0118733374401927, 0.012630254030227661, 0.011918239295482635, 0.007973289117217064, -0.009647487662732601, 0.01834562048316002, -0.002833628561347723, -0.019782479852437973, 0.011340929195284843, 0.019089708104729652, 0.014201819896697998, 0.0023236717097461224, 0.010186309926211834, 0.023977598175406456, 0.0018618239555507898, 0.005205409601330757, 0.033637914806604385, 0.013662997633218765, 0.00544916233047843, -0.015510388650000095, 0.03186749666929245, -0.005378602538257837, -0.03235500305891037, 0.013752801343798637, 0.0006743298727087677, -0.004426041152328253, -0.012495548464357853, 0.010256869718432426, 0.008531355299055576, 0.0034317856188863516, -0.0025112973526120186, -0.07861676067113876, -0.007062422577291727, 0.0063311634585261345, 0.02537596970796585, -0.02704375423491001, 0.027864817529916763, 0.004207946360111237, 0.013329440727829933, -0.01579262875020504, 0.01815318502485752, 0.007120153401046991, -0.02359272539615631, -0.02627400867640972, -0.013752801343798637, 0.015459071844816208, 0.029301676899194717, 0.0025161083322018385, -0.003045308869332075, -0.017601532861590385, -0.0025818573776632547, -0.009910483844578266, -0.010410819202661514, -0.011167735792696476, 0.0014865725534036756, -0.009756534360349178, 0.009429392404854298, -0.014278794638812542, 0.007960460148751736, 0.007152226287871599, -0.001069626654498279, 0.018576543778181076, -0.012874007225036621, 0.002876926911994815, -0.04015510156750679, -4.9462214519735426e-05, -0.0053080422803759575, 0.007819339632987976, -0.001986907795071602, 0.012354428879916668, -0.01244423259049654, 0.01266874186694622, 0.0017367401160299778, -0.011693730019032955, -0.047108475118875504, 0.020513739436864853, -0.013855434022843838, -0.007004691753536463, 0.005785926710814238, 0.029455626383423805, -0.03610110282897949, -0.005981570575386286, 0.00186984206084162, -0.013791288249194622, -0.0074537103064358234, 0.014933078549802303, -0.02078315056860447, 0.012315941043198109, -0.025812160223722458, -0.020026233047246933, 0.007883485406637192, -0.011270369403064251, -0.020885784178972244, -0.023772332817316055, 0.0340997613966465, 0.007941216230392456, 0.013470560312271118, -0.025914791971445084, -0.050854574888944626, 0.0158311165869236, -0.01140507496893406, -0.01242498867213726, -0.007203542627394199, -0.014522546902298927, -0.006959789898246527, -0.015407755970954895, -0.01896141842007637, -0.024708857759833336, 0.0023733845446258783, 0.020847296342253685, -0.029045093804597855, -0.010167066007852554, -0.021283484995365143, 0.010391575284302235, -0.018640689551830292, 0.012104260735213757, 0.018012063577771187, 0.012072187848389149, 0.009878410957753658, -0.012392915785312653, -0.01875615119934082, 0.009038104675710201, -0.0017110819462686777, 0.03325304016470909, -0.020654859021306038, -0.02087295427918434, 0.02010320872068405, -0.007235615514218807, -0.01566433720290661, 0.013778459280729294, -0.023823648691177368, -0.013290952891111374, 0.008775108493864536, -0.044593971222639084, 0.0067032077349722385, -0.018089039251208305, -0.015369268134236336, 0.013329440727829933, -0.008563428185880184, -0.00937807559967041, 0.005975155625492334, 0.0062926760874688625, 0.015561704523861408, -0.012803447432816029, 0.009576926939189434, 0.010372331365942955, -0.001885878504253924, 0.0023894209880381823, -0.005628770217299461, 0.02927601896226406, 0.014214648865163326, 0.01672915369272232, -0.001007084734737873, 0.019795309752225876, 0.013675826601684093, -0.0036242222413420677, -0.0113665871322155, 0.003111058147624135, 0.008845668286085129, -0.005073911044746637, 0.028531929478049278, -0.010391575284302235, -0.0194874107837677, 0.019064050167798996, -0.019269315525889397, -0.012578938156366348, 0.032457634806632996, 0.0022819770965725183, -0.02359272539615631, -0.003890426130965352, 0.015266635455191135, -0.0046954527497291565, 0.013079272583127022, -0.0031190761364996433, -0.02655624784529209, 0.019679848104715347, -0.025914791971445084, -0.014817616902291775, -0.014022212475538254, -0.008685304783284664, 0.00998745858669281, -0.005138056818395853, 0.01599789410829544, 0.029096411541104317, 0.011430732905864716, 0.013470560312271118, -0.03779454529285431, 0.0014408689457923174, -0.02139894850552082, 0.010564768686890602, 0.02732599526643753, 0.008787937462329865, 0.001094483071938157, 0.02256639674305916, 0.006472283508628607, -0.025145046412944794, -0.016536716371774673, -0.024991096928715706, -0.010956056416034698, -0.010244040749967098, 0.0061643850058317184, 0.023310484364628792, -0.02789047546684742, -0.022438105195760727, 0.007934801280498505, 0.0033034945372492075, -0.007524270098656416, 0.018935760483145714, 0.003867975203320384, 0.007267688401043415, 0.001003877492621541, -0.0003912877000402659, 0.0442604124546051, 0.019923601299524307, 0.026812830939888954, -0.020885784178972244, 0.03758927807211876, 0.0017784347292035818, -0.0034702729899436235, 0.0004931187140755355, 0.0014713379787281156, 0.01404787041246891, 0.0005945488810539246, 0.016139013692736626, 0.0022739588748663664, 0.0276082344353199, -0.0026524176355451345, 0.020924270153045654, 0.016190331429243088, -0.007299760822206736, -0.027556918561458588, 0.00996821466833353, 0.018076209351420403, 0.019743993878364563, -0.010442892089486122, -0.01708836853504181, -0.013483390212059021, -0.011905409395694733, -0.014060699380934238, -0.04857099428772926, -0.018858784809708595, -0.025606894865632057, 0.0043234084732830524, 0.025812160223722458, 0.010507036931812763, 0.016382766887545586, 0.030071422457695007, -0.01530512236058712, 0.004288128577172756, -0.02590196393430233, -0.002894566860049963, 0.00810157973319292, 0.0013117759954184294, 0.002196984365582466, 0.015292293392121792, 0.009564097970724106, -0.0024022499565035105, -0.007081666029989719, 0.002812781371176243, 0.002102369675412774, -0.03096945956349373, 0.015420584939420223, 0.009589755907654762, -0.0064754909835755825, -0.0073318337090313435, -0.002026998670771718, -0.015394926071166992, -0.02712072804570198, 0.00927544292062521, 0.015035711228847504, 0.020167354494333267, -0.019820967689156532, 0.04213078320026398, 0.01640842668712139, 0.010244040749967098, -0.002860890468582511, 0.021078219637274742, 0.01858937367796898, 0.0047179036773741245, -0.013162662275135517, -0.014894591644406319, -0.013053614646196365, 0.004884681664407253, 0.014407085254788399, 0.007594830356538296, -0.001111321267671883, -0.027582576498389244, -0.0024455483071506023, -0.021655529737472534, -0.007505026645958424, -0.009429392404854298, -0.016177501529455185, 0.0072484444826841354, 0.017139684408903122, 0.014561034739017487, 0.007075251545757055, -0.02289995364844799, 0.006411345209926367, 0.018768981099128723, 0.0026347774546593428, -0.008871326223015785, -0.03989851847290993, 0.008204213343560696, 0.013060029596090317, -0.03299646079540253, -0.015536046586930752, 0.02464471198618412, -0.02894246205687523, -0.006485112942755222, 0.008768693543970585, 0.016831785440444946, 0.035664912313222885, -0.0032938728109002113, 0.022425275295972824, -0.02050090953707695, -0.00025557982735335827, 0.013239637017250061, -0.016652178019285202, 0.004143801052123308, -0.0032072763424366713, -0.025273337960243225]

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    query_properties=["question^2", "answer"],
    return_properties=["question", "answer"],
    vector=query_vector,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "pasta",
  "question": "Type of food that comes in shapes of bow ties, elbows & wagon wheels"
}
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "devilled",
  "question": "Satanic term for food prepared with strong seasoning, as of eggs or ham"
}


In [41]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.hybrid(
    query="food",
    filters=wvc.Filter("round").equal("Double Jeopardy!"),
    return_properties=["question", "answer"],
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "Tofu",
  "question": "A popular health food, this soybean curd is used to make a variety of dishes & an ice cream substitute"
}
{
  "answer": "play on",
  "question": "\"Twelfth Night\" opens with, \"If music be the food of love,\" do this"
}


In [45]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    query="animals in movies",
    return_properties=["question", "answer"],
    limit=2
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    

{
  "answer": "meerkats",
  "question": "Group of mammals seen <a href=\"http://www.j-archive.com/media/1998-06-01_J_28.jpg\" target=\"_blank\">here</a>:  [like Timon in <i>The Lion King</i>]"
}
{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}


In [48]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_object(
    near_object="56b9449e-65db-5df4-887b-0a4773f52aa7",
    return_properties=["question", "answer"],
    limit=2
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}
{
  "answer": "Tiny Toon Adventures",
  "question": "On this cartoon series, you'll find such characters as Plucky Duck, Dizzy Devil & Buster Bunny"
}


In [51]:
query_vector = [-0.0125526935, -0.021168863, -0.01076519, -0.02589537, -0.0070362035, 0.019870078, -0.010001986, -0.019120263, 0.00090044655, -0.017393013, 0.021302758, 0.010055545, 0.02937665, -0.003816019, 0.007692291, 0.012385325, 0.032750815, 0.020847514, 0.020311933, -0.022159688, -0.0009924996, 0.009399457, 0.0022226637, -0.029510546, 0.014393755, -0.007223657, 0.018276723, -0.03639277, -0.010001986, -0.022842556, 0.010363504, -0.020927852, -0.006929087, -0.022521207, -0.007652122, -0.011126708, 0.0279038, -0.01721895, 0.016482525, 0.002281243, -0.00169294, 0.009191919, -0.019655844, -0.022869334, -0.012412104, 0.0031967526, -0.0033457114, -0.01483561, -0.03173321, 0.004746592, 0.010095714, 0.007973471, -0.032134898, -0.023739655, -0.008040419, 0.018290112, -0.013637247, -0.008488968, 0.024623364, -0.039365247, -0.0032586793, 0.0009606995, -0.029510546, 0.0063265576, -0.019602288, 0.003081268, 0.013463182, -0.006601043, 0.019910246, -0.01542475, 0.0367409, -0.01193008, 0.012961075, -0.015625594, 0.0062462203, -0.0058646183, -0.0059248717, 0.01889264, 0.008127451, 0.0037155973, 0.037142586, -0.025373178, -0.005503101, 0.014982895, 0.035053816, -0.012432188, -0.017285896, 0.022936283, 0.0024620018, 0.016937768, -0.0062127467, 0.02154377, 0.0066378643, 0.029698, 0.0013071538, 0.0043850746, -0.008040419, 0.024797428, -0.012452273, -0.025132166, -0.0031900578, 0.0000019433794, -0.002378317, -0.008629559, 0.0126732, -0.0022494427, 0.0009623732, 0.0035582704, 0.017312676, -0.024569806, -0.008890655, 0.023056788, 0.014902558, -0.047104403, -0.009011161, -0.030447815, 0.017982153, -0.0042009684, -0.00654079, 0.00069249026, 0.011936775, 0.023378137, 0.025105387, -0.009245478, 0.030929837, 0.00394322, 0.02123581, -0.0042545265, 0.0022578111, -0.017259117, 0.047157962, -0.00022029977, 0.03497348, -0.00072094303, -0.023605758, 0.036499888, -0.015384582, 0.011099929, -0.0139519, -0.03408977, 0.013155223, 0.030501373, -0.026698742, 0.004311432, -0.010236303, 0.011361024, 0.023793213, -0.00014874942, 0.0020352101, 0.0026829292, 0.00989487, 0.0074780583, 0.02734144, 0.003826061, 0.011722542, 0.00712993, -0.013992069, 0.0009406152, 0.010785274, -0.012325072, 0.01692438, 0.010617905, 0.016750315, -0.0070295087, 0.017687583, 0.038320865, 0.020485997, 0.005054551, -0.018812304, 0.0007201062, 0.0015381235, 0.0349467, 0.014728494, 0.050773136, -0.017901815, 0.0027716348, 0.0064704954, 0.026671965, -0.015063233, -0.013536825, 0.016696757, 0.008127451, 0.026966535, 0.029912233, -0.0031431946, 0.015156959, 0.012412104, -0.047907773, 0.022012403, -0.027006702, -0.0069491714, 0.010718327, 0.011976943, -0.008127451, -0.65212417, 0.00024289463, 0.0051214993, -0.013007938, 0.022373922, 0.0337952, -0.0026829292, -0.0110463705, -0.013034717, -0.0012167745, 0.010062239, -0.0023013272, 0.024409132, -0.009118277, -0.020191427, -0.01597372, 0.010115798, -0.030929837, -0.010932559, 0.010912475, -0.0009841312, 0.010571042, -0.008348378, -0.009104887, 0.02711382, 0.0036553445, -0.018263333, -0.030876279, 0.014594599, 0.037704945, -0.030126465, 0.014366977, 0.0055533117, 0.003487975, 0.044988856, 0.009881481, -0.012699978, 0.041132666, 0.01744657, 0.05417408, -0.004686339, 0.016121006, 0.0070495927, 0.015478308, -0.020593112, 0.0012376956, 0.027127208, -0.0051248465, 0.0005979267, 0.0063366, -0.008616169, 0.027877023, -0.00042679158, 0.008442105, 0.00069751136, 0.023806602, 0.029296314, -0.0047332025, 0.027877023, 0.0033005215, 0.014996285, -0.0061424514, 0.00451897, 0.015531867, -0.015317634, 0.044185482, 0.010196134, 0.007504837, 0.012405409, -0.030126465, 0.03821375, 0.0256008, -0.016710145, 0.0032804373, -0.013884953, 0.022775607, 0.030608488, -0.023431696, -0.008502358, 0.008683117, -0.0045490963, -0.0030143203, -0.024074392, 0.00874337, 0.009466405, -0.0072370465, -0.021383096, 0.001360712, 0.020298542, 0.0040168623, 0.008201093, 0.011106623, -0.03202778, 0.0046461704, -0.00088370964, -0.008957602, 0.0057575023, 0.00037407028, 0.017259117, -0.0482559, -0.0049507823, -0.024235068, -0.0014418861, 0.004425243, 0.023244241, 0.0107919695, -0.017058274, 0.0183035, 0.033339955, -0.009091497, 0.000118936776, 0.0031900578, -0.000044483608, -0.017058274, 0.001529755, -0.027984139, 0.02740839, -0.015344413, 0.015264076, -0.01719217, 0.010463926, -0.0067048124, 0.014942727, -0.00026653553, 0.02677908, -0.00036570182, -0.043194655, -0.022855945, -0.011294077, 0.005764197, 0.004910614, -0.0029724778, 0.0056637754, -0.01425986, -0.000008708432, 0.01866502, 0.031626094, 0.0050378144, 0.015451529, 0.009406152, -0.030742384, -0.0024318753, -0.029751558, -0.008348378, 0.0028519721, -0.008388547, -0.010611211, 0.0139519, -0.0006895613, -0.001230164, -0.0062462203, -0.013510046, 0.010617905, -0.010229609, 0.022213247, -0.00610563, -0.00568386, -0.0056503857, 0.02416812, -0.0076253433, 0.015183738, -0.005188447, -0.016080838, 0.013516741, 0.0062897364, -0.0068520973, 0.021396484, 0.007799407, -0.01721895, -0.025266062, 0.013791226, -0.017205559, -0.002068684, 0.032938268, 0.014661547, 0.023552202, -0.005827797, -0.008442105, -0.0074914475, 0.009111582, 0.016817262, -0.0050244248, -0.005871313, -0.008368462, 0.040329296, 0.008683117, 0.031518977, 0.026109602, -0.025815032, 0.011006202, -0.0034310697, 0.019575508, -0.013831395, -0.008676422, -0.008770149, -0.019990584, 0.008750064, 0.02851972, 0.0337952, 0.012666505, 0.021383096, -0.027448557, 0.0035448808, -0.016214734, 0.015197128, -0.027582452, -0.0138046155, -0.03899034, 0.008261346, 0.015478308, 0.017888425, 0.0153979715, 0.010658074, -0.011581952, 0.02530623, 0.017982153, -0.0059449556, 0.0054294583, 0.0022879376, -0.018758746, -0.0076119537, -0.027689569, 0.013463182, 0.011186961, -0.0063165156, 0.028412605, 0.011347636, 0.008709895, -0.003374164, -0.007919913, -0.025828423, 0.0033875536, -0.013831395, -0.0035716598, 0.010450536, -0.025172336, 0.003990083, -0.00093224674, 0.024047613, 0.008027029, -0.0029440252, 0.023458473, 0.016643198, -0.0326437, 0.019147042, 0.01925416, -0.0020151257, 0.0038628823, -0.026738912, 0.0008753412, -0.025105387, 0.0069491714, -0.02623011, 0.027033482, -0.0040737675, -0.021034967, 0.019468391, 0.0026042655, 0.03467891, 0.016107617, -0.0057139862, -0.011735932, 0.017687583, 0.011628816, 0.015090012, -0.006678033, -0.011715848, -0.01833028, 0.008040419, -0.01921399, -0.03267048, -0.005914829, 0.0014435598, -0.0030662047, 0.005479669, 0.01597372, -0.01454104, 0.023257632, 0.019722793, 0.0344379, 0.006929087, -0.043248214, 0.015853215, 0.012766927, -0.007417805, -0.018316891, -0.01163551, -0.017352844, -0.01978974, 0.015304244, -0.00005920687, 0.033580966, -0.0022343795, 0.0047800657, -0.007357552, 0.00033536615, 0.00887057, -0.025654359, 0.016388796, -0.011361024, 0.00019090556, 0.0060119033, -0.010075629, -0.0131485285, 0.01604067, -0.015531867, 0.0035616176, -0.017259117, 0.0035415334, 0.009265562, -0.0043348637, -0.005867966, -0.03283115, -0.004773371, -0.018410617, -0.0095400475, -0.006520706, -0.00414741, 0.031197628, 0.013690805, -0.008984381, -0.022320364, -0.012492441, -0.005724028, 0.09806499, 0.017272506, -0.00007704216, 0.00858939, 0.0030126465, -0.002835235, -0.023753043, -0.025587412, 0.016067449, 0.0024536331, 0.004719813, -0.02908208, 0.027743127, 0.0023414958, 0.0152908545, 0.00552988, -0.031974223, 0.0019582203, 0.010812053, -0.01952195, -0.00006171741, -0.02241409, 0.025252672, 0.013737668, 0.002356559, -0.03719614, 0.021637497, 0.033580966, 0.0044453274, -0.0074378895, -0.014715104, -0.01741979, -0.013489962, -0.003221858, 0.0038561875, -0.013121749, -0.012974464, 0.012619642, 0.053424265, -0.020459218, 0.011581952, 0.041962817, -0.00087032013, -0.0036988605, -0.0010025419, -0.020392269, 0.014902558, 0.021409875, 0.01771436, -0.006483885, 0.036633782, -0.00028808432, 0.011983639, 0.014326808, 0.024931323, 0.002629371, -0.01223804, -0.010972728, -0.011253908, 0.013831395, -0.01748674, -0.013777837, -0.0043449057, -0.009292341, -0.0015849868, -0.019455003, -0.031170849, -0.014393755, -0.03778528, -0.0028335615, -0.00785966, -0.027528895, -0.021008188, -0.03786562, -0.0008226199, -0.005539922, 0.011970249, -0.016937768, -0.0044553694, 0.015839826, -0.014929337, -0.011166876, 0.0031448682, -0.032402687, -0.011207045, -0.009432931, 0.0034059642, -0.00089124124, -0.009439626, -0.012840569, 0.013610467, 0.008877265, 0.006108978, 0.0021289368, 0.039124236, 0.0025557284, -0.004277958, 0.02822515, 0.022373922, -0.00888396, 0.032777593, -0.021610718, -0.010490704, -0.0017222296, -0.011113319, -0.024569806, 0.0024703701, 0.021155473, -0.004555791, -0.0060353354, 0.008241262, -0.03234913, -0.00048076818, -0.0069960346, 0.02910886, 0.013315897, -0.014728494, 0.01454104, -0.00567047, -0.0012602905, 0.0001736456, 0.005302258, -0.0000424961, 0.035589397, -0.01570593, 0.0107919695, 0.0051348885, -0.015331023, -0.0034193539, 0.003625218, -0.010477315, 0.024583196, -0.0030226887, -0.011776101, -0.040115062, -0.009091497, -0.003886314, 0.017888425, -0.03143864, -0.008629559, -0.005533227, -0.017138612, 0.01338954, -0.02681925, -0.006688075, -0.026538068, 0.0050210776, 0.011401193, 0.0076655117, 0.008576, -0.028171593, -0.0022025793, 0.005911482, 0.017205559, -0.02066006, -0.0413469, -0.016910989, 0.0097944485, 0.020807344, 0.030742384, 0.026738912, -0.011628816, 0.03350063, 0.011146792, -0.024556417, 0.019709403, -0.00712993, 0.012110839, -0.044694286, 0.02795736, 0.016777094, -0.0054729744, 0.025975708, 0.0109191695, 0.009821228, 0.012485746, 0.01571932, 0.0018661672, -0.014567819, -0.010972728, 0.0022394005, 0.01626829, 0.0014820547, -0.0030026045, 0.004120631, -0.023699487, 0.040918436, 0.0011640531, -0.0092856465, -0.0180491, 0.03459857, -0.013161918, -0.0036151758, -0.0073910262, 0.0028737301, -0.017968763, -0.016549472, -0.01355691, 0.0031616052, 0.0067516756, 0.0023096956, -0.0076789013, -0.009955123, 0.011233824, -0.0072906045, 0.016402187, 0.009727501, -0.0153979715, 0.020445827, -0.0042980425, -0.024556417, -0.048496913, -0.026886197, -0.047693543, 0.0007615301, -0.013925122, -0.010437147, 0.01483561, -0.0050277724, -0.022266805, 0.02793058, -0.015264076, 0.032563362, 0.00472316, 0.017526908, 0.021061746, -0.013818005, -0.021945456, 0.028573278, -0.0313583, 0.016469134, 0.00013180329, -0.000116426236, -0.0018477566, -0.03722292, -0.002868709, 0.001186648, -0.037463933, -0.046568822, 0.0128004, 0.015197128, 0.013054801, -0.017821478, -0.022320364, -0.022012403, 0.013289118, -0.0043516005, -0.0029808464, -0.01660303, -0.03786562, -0.024877766, -0.013356066, -0.006825318, 0.027582452, -0.0042545265, -0.0017063295, 0.024891155, -0.0049240035, -0.014500872, -0.016803874, 0.008127451, 0.022855945, -0.0014284966, -0.006339947, 0.01604067, 0.0026092867, 0.012057281, -0.008569306, 0.00007374708, 0.02766279, -0.025774864, 0.0047064233, -0.024676923, 0.013938512, -0.002286264, -0.011166876, -0.024074392, -0.018450785, -0.0049842563, 0.0035080595, 0.028305488, 0.033286396, -0.003054489, -0.003272069, -0.024502859, 0.021302758, -0.015558646, -0.006798539, 0.005667123, -0.01716539, 0.003325627, 0.00885718, -0.0047767186, -0.0073843314, -0.0038193662, -0.009352594, 0.0209948, 0.041507576, -0.036526665, -0.0022661798, -0.035401944, 0.012204566, -0.034759246, -0.008850486, -0.0009975208, -0.00022176426, -0.008629559, -0.015357803, -0.01455443, -0.0059416085, -0.01687082, 0.014487483, -0.0008845465, -0.0010284841, 0.02708704, 0.028653616, 0.0033189321, -0.025373178, 0.0036620393, 0.018772135, -0.0031130682, 0.0070495927, -0.00006830758, -0.017674193, 0.000969068, -0.018290112, -0.005546617, 0.0037658082, -0.00016872912, -0.024784038, -0.020860903, 0.02070023, 0.0029138986, -0.036285654, -0.041159447, -0.022106132, -0.018651629, 0.03435756, -0.008194398, -0.020485997, 0.01660303, 0.026270278, 0.0079065235, 0.0015649025, -0.005807713, -0.012733453, -0.0042377897, -0.021891898, -0.0180491, -0.008783538, -0.017111832, 0.005493059, 0.011501615, -0.0025657706, -0.018946199, 0.006052072, -0.0120438915, 0.010644685, -0.005165015, 0.009881481, 0.02677908, -0.0035716598, 0.005449543, 0.021758003, -0.0072035724, 0.010745106, -0.012130924, -0.0011799532, 0.0036620393, -0.0034411119, 0.013028023, 0.045095973, -0.021396484, -0.01895959, 0.016281681, 0.0020050837, 0.008214483, 0.004632781, -0.030501373, -0.019709403, -0.021075137, -0.0027230978, -0.015183738, 0.0008828728, 0.015304244, -0.0034578487, -0.02940343, 0.015344413, 0.00785966, -0.0026260235, -0.008529137, 0.00442859, 0.0013900016, 0.0001500047, -0.024368962, -0.005580091, -0.017205559, -0.0285465, 0.0054729744, -0.0009422889, -0.0076722065, 0.02475726, -0.02241409, -0.016469134, -0.0064370213, 0.00018034037, 0.009044634, -0.0044486746, 0.000060462142, -0.014942727, 0.026658574, -0.0043181265, 0.030046128, -0.042043157, 0.016616419, -0.007170099, 0.02040566, -0.008227873, 0.025975708, -0.027877023, -0.022668492, 0.0051181517, -0.007116541, 0.016522693, -0.0025373178, -0.0018259985, -0.015906774, 0.013858174, -0.019843299, 0.0029942358, -0.01632185, -0.029831896, -0.024007445, -0.0045022327, -0.015946941, 0.030662047, 0.18091947, -0.016576251, 0.003936525, 0.039659817, -0.008160925, 0.021168863, 0.026002487, -0.0043248213, -0.008488968, 0.0125526935, -0.007839575, 0.024020836, -0.014500872, 0.008529137, 0.0011925059, -0.015652372, -0.00050880254, -0.0032017739, -0.006353337, -0.03438434, -0.013208781, -0.0023113694, -0.011608731, -0.015411361, 0.022842556, 0.0013423014, -0.0017356192, -0.005104762, 0.0062395255, 0.0056403438, 0.0061960095, -0.033018608, 0.0053591635, -0.02067345, -0.001453602, -0.013289118, -0.02851972, 0.028118035, 0.0052687842, 0.01338954, -0.0035314912, 0.009673943, 0.009191919, 0.01281379, -0.013992069, 0.008134145, -0.004575875, 0.0015013022, -0.00028620142, 0.03550906, -0.0512016, 0.010477315, 0.008897349, 0.03347385, -0.02471709, 0.0011297425, 0.005851229, -0.019588897, 0.012037196, 0.010182745, 0.0065776114, 0.030233582, -0.01309497, 0.018839084, -0.024623364, 0.0072370465, -0.02241409, 0.03400943, -0.00069207186, -0.014674936, -0.0031833632, -0.024784038, -0.02645773, -0.012793706, -0.0008506542, -0.03583041, -0.012325072, 0.026966535, 0.01018944, 0.013356066, -0.02474387, -0.014326808, -0.007658817, 0.012827179, -0.02740839, -0.015277465, 0.021784782, -0.0015858236, -0.0018460829, 0.0004573365, -0.0057072914, -0.019588897, 0.0058411867, -0.002308022, -0.00066278223, 0.006460453, -0.00038369402, 0.018705187, -0.009078108, -0.020298542, -0.035991084, -0.047211517, 0.018571293, -0.0041775363, -0.008676422, -0.002138979, -0.007504837, -0.00078579865, 0.014621378, -0.0043850746, -0.01455443, -0.015906774, 0.0010176051, 0.006935782, 0.025199115, -0.0038093242, 0.013690805, -0.022253416, 0.036874793, -0.019053316, -0.0044821487, 0.0042377897, 0.005998514, 0.0064102425, 0.008080588, -0.028064476, -0.025239283, 0.0070295087, 0.023083568, -0.028653616, -0.010771885, -0.019280938, -0.005563354, -0.012579473, -0.005258742, 0.0012109166, 0.015531867, -0.017339455, 0.016241511, 0.0069424766, 0.015652372, 0.014380367, 0.006791844, -0.0023967277, 0.037945956, -0.0285465, 0.02128937, 0.0049942983, -0.029831896, -0.023819992, -0.016281681, -0.0031850368, 0.0029691304, -0.0038227136, 0.023645928, -0.036473107, -0.02153038, -0.025279451, -0.010242999, 0.018156216, -0.025413347, 0.0036218707, 0.005111457, -0.014487483, -0.0059784297, -0.013690805, -0.171279, -0.0037222921, 0.01626829, -0.010417062, -0.0007322405, -0.001834367, 0.008776844, -0.012867348, -0.005884703, -0.0027147292, 0.022306973, 0.0042244, -0.049300287, -0.0157461, 0.016054058, 0.002781677, 0.00197161, 0.007980166, -0.014366977, -0.0071834885, 0.021048358, -0.024971493, 0.017955374, -0.007692291, 0.0043683373, 0.018557902, 0.01570593, 0.0027063608, 0.0011791164, -0.03698191, -0.014875779, 0.008455494, 0.016536081, 0.009486489, -0.001415107, 0.002960762, -0.008368462, -0.021878509, -0.022454258, 0.004686339, 0.012392019, 0.04394447, 0.016121006, -0.0068085813, 0.014085797, -0.0022946324, 0.008509053, -0.0063868104, 0.022333752, -0.026591627, 0.006497274, -0.01454104, 0.0080337245, -0.0059014396, 0.01602728, 0.02651129, -0.010738411, 0.014567819, -0.010303251, -0.031010175, -0.03821375, -0.0056403438, -0.00006835988, -0.0011732584, -0.021945456, -0.011146792, -0.023498643, 0.021409875, -0.026712133, -0.004190926, 0.002542339, 0.0062462203, -0.004522317, -0.02967122, 0.008334989, 0.00029415145, -0.018544514, 0.022240026, -0.024261847, -0.021811562, -0.020566333, 0.0390439, -0.025466906, 0.014059017, 0.013476572, -0.007451279, -0.0101760505, -0.021918677, 0.004093852, -0.003772503, 0.034304, -0.029483767, -0.025574021, -0.015893385, -0.003407638, 0.030233582, 0.007799407, 0.00002280406, 0.021838339, -0.01633524, -0.006875529, -0.010229609, 0.0053256894, 0.02011109, -0.010885696, 0.04016862, 0.028760733, 0.015183738, 0.013061496, -0.0073307734, 0.0077324593, 0.007739154, 0.015344413, 0.03783884, -0.012124228, 0.0145276515, -0.00086027797, 0.0006744981, 0.035375167, -0.0044620642, 0.030903058, 0.01567915, -0.0053189946, -0.014045628, -0.018852472, 0.0035683124, -0.09554776, -0.013791226, -0.015116791, 0.0013891648, -0.026377395, 0.019147042, -0.008254652, 0.040623866, -0.01656286, 0.01948178, 0.01310836, -0.006025293, 0.005971735, -0.0051348885, 0.019843299, 0.02007092, -0.027421778, 0.00007709446, 0.0038896615, 0.005737418, 0.010095714, -0.0044988855, 0.011294077, -0.001899641, -0.01567915, 0.007216962, -0.02095463, 0.024797428, -0.0064805374, 0.010691548, 0.01208406, -0.012867348, -0.0057775867, -0.023110347, -0.019588897, 0.0060821986, -0.019374665, -0.0061391043, 0.031331524, -0.018490955, 0.004043641, 0.0032017739, -0.003973346, -0.014982895, 0.008696507, -0.025989097, 0.007156709, 0.013523435, -0.0041139363, -0.03055493, -0.02793058, -0.011106623, -0.02851972, 0.023753043, 0.04689017, 0.0035850494, 0.009834617, 0.0096003, 0.016147785, 0.019856688, 0.0031582578, 0.004666255, -0.00829482, 0.0395527, -0.01077858, -0.020512775, -0.020512775, 0.012057281, 0.027006702, -0.021999015, -0.009633774, 0.02878751, -0.026645185, -0.005057899, -0.016964547, 0.003315585, -0.02910886, -0.008107367, 0.0138046155, -0.023538811, -0.0028804247, -0.03491992, 0.0076789013, -0.03781206, 0.014032238, 0.019642456, 0.021798171, -0.0074780583, -0.01602728, -0.011909996, -0.015183738, 0.0031063734, 0.0016686714, -0.036553446, -0.0018594724, 0.015906774, -0.009225393, 0.006755023, 0.0065776114, 0.0139117325, -0.0045524435, 0.0051583205, -0.049166393, 0.018477565, -0.010182745, 0.0031398472, 0.022614934, 0.0048905294, 0.027234325, -0.005191794, 0.026966535, -0.0012477378, -0.029483767, 0.010303251, -0.0072370465, 0.015505088, -0.015183738, -0.009948429, 0.00054185797, -0.016844042, 0.0015339392, -0.008495663, 0.01105976, 0.008375158, 0.013992069, 0.00698934, 0.0035448808, 0.01427325, 0.0080538085, 0.005382595, -0.021677665, 0.004900572, 0.008977687, -0.034812804, 0.005998514, 0.024984881, 0.0032687215, -0.02795736, 0.009124972, 0.0022778956, 0.0038126716, 0.012646421, 0.0019180516, -0.0128004, 0.013034717, -0.046033237, -0.00021506949, -0.005104762, -0.010309946, 0.0054093744, 0.01632185, -0.005737418, 0.016937768, 0.010945949, -0.018129438, 0.0039532618, -0.0047432445, -0.04051675, 0.03703547, 0.007551701, 0.0031264576, 0.00073935365, 0.012887432, 0.00020000625, 0.003869577, -0.012961075, -0.010443841, 0.038481537, 0.0037089025, -0.013643941, 0.03639277, -0.040329296, -0.022293584, 0.004087157, 0.011709153, 0.014902558, -0.006122367, 0.007852965, 0.003081268, 0.018571293, -0.0077190697, 0.020927852, 0.021195643, -0.00010554723, -0.029055303, 0.006269652, 0.029885454, 0.0060554193, -0.0075583956, 0.0008188541, 0.013041412, -0.006453758, -0.03467891, 0.005814408, 0.015090012, 0.010383588, 0.013818005, 0.018504344, -0.025761476, -0.011856438, 0.0052219206, 0.021423263, 0.00829482, -0.009881481, -0.01326234, -0.0038093242, -0.016951159, -0.005590133, -0.0067115068, -0.03483958, -0.010838833, 0.01717878, 0.038053073, -0.015612204, -0.004231095, 0.008027029, -0.008040419, 0.025667747, -0.005677165, -0.016455745, -0.028010918, 0.024342183, 0.010095714, 0.014219692, 0.016910989, -0.00083266204, 0.010182745, 0.021516992, 0.011950164, -0.03055493, -0.012104144, 0.004468759, -0.006969256, -0.014393755, -0.021342928, 0.0085626105, -0.015665762, 0.0021841687, -0.004234442, 0.030715605, -0.017138612, 0.042712633, 0.0062462203, 0.020057531, 0.008073892, -0.0326437, 0.01250583, 0.024342183, 0.04747931, -0.027020091, 0.0019414834, 0.015170349, -0.016362019, 0.02825193, -0.009566827, -0.039954387, -0.00697595, -0.023927107, -0.0285465, -0.000100316945, -0.024850987, 0.022963062, 0.002122242, 0.027877023, 0.0012870695, -0.018182995, -0.0079266075, 0.016174564, -0.010068934, -0.015090012, -0.0054227635, 0.0051516257, -0.013235561, -0.0075583956, -0.0131485285, 0.039365247, 0.0065575275, -0.011474836, 0.0028268667, -0.004425243, -0.0020703576, -0.010631295, -0.011702458, -0.0038394507, 0.0059784297, 0.032268792, 0.02244087, -0.023458473, -0.0053859423, -0.01925416]

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_vector(
    near_vector=query_vector, # your query vector goes here
    return_properties=["question", "answer"],
    limit=2
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "meerkats",
  "question": "Group of mammals seen <a href=\"http://www.j-archive.com/media/1998-06-01_J_28.jpg\" target=\"_blank\">here</a>:  [like Timon in <i>The Lion King</i>]"
}
{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}


In [53]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    query="animals in movies",
    # offset=1, # offset is currently not supported with the Weaviate Client v4
    return_properties=["question", "answer"],
    limit=2
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "meerkats",
  "question": "Group of mammals seen <a href=\"http://www.j-archive.com/media/1998-06-01_J_28.jpg\" target=\"_blank\">here</a>:  [like Timon in <i>The Lion King</i>]"
}
{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}


In [56]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    query="animals in movies",
    distance=0.18, # max accepted distance
    return_properties=["question", "answer"],
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "meerkats",
  "question": "Group of mammals seen <a href=\"http://www.j-archive.com/media/1998-06-01_J_28.jpg\" target=\"_blank\">here</a>:  [like Timon in <i>The Lion King</i>]"
}
{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}


In [58]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    query="animals in movies",
    auto_limit=1, # number of close groups
    return_properties=["question", "answer"],
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "meerkats",
  "question": "Group of mammals seen <a href=\"http://www.j-archive.com/media/1998-06-01_J_28.jpg\" target=\"_blank\">here</a>:  [like Timon in <i>The Lion King</i>]"
}
{
  "answer": "dogs",
  "question": "Scooby-Doo, Goofy & Pluto are cartoon versions"
}


In [59]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate_group_by.near_text(
    query="animals in movies", # find object based on this query
    object_limit=100,          # fetch top 100 best search results
    group_by="round",          # then group them by round
    limit=2,                   # keep only top 2 groups
)

# print the group names (as per group_by) and
#       the total count (out of 100 as per object_limit)
for group in response:
    print (group.grouped_by.value, group.total_count)

Jeopardy! 57
Double Jeopardy! 41


In [62]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    query="animals in movies",
    filters=wvc.Filter("round").equal("Double Jeopardy!"),
    return_properties=["question", "answer"],
    limit=2, # number of close groups
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "fox",
  "question": "In titles, animal associated with both Volpone and Reynard"
}
{
  "answer": "Swan",
  "question": "In a Tchaikovsky ballet, Prince Siegfried goes hunting for these animals & falls in love with 1 of them"
}


In [64]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    return_properties=["question", "answer"],
    query="food",
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert"
}
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}


In [65]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    return_properties=["question", "answer"],
    query="food",
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert"
}
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}


In [68]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="food",
    return_metadata=wvc.MetadataQuery(score=True),
    return_properties=["question", "answer"],
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    print(o.metadata.score)

{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
3.045816421508789
{
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert"
}
2.9159791469573975
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
2.803023338317871


In [69]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="safety",
    return_properties=["question", "answer"],
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "OSHA (Occupational Safety and Health Administration)",
  "question": "The government admin. was created in 1971 to ensure occupational health & safety standards"
}
{
  "answer": "France",
  "question": "Royale, Joseph, and Devil's Islands make up the Safety Islands owned by this country"
}
{
  "answer": "Devil's Island",
  "question": "The Safety Islands off French Guiana consist of Royale, Saint-Joseph & this diabolical island"
}


In [71]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    return_properties=["question", "answer"],
    query="safety",
    auto_limit=1
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "OSHA (Occupational Safety and Health Administration)",
  "question": "The government admin. was created in 1971 to ensure occupational health & safety standards"
}


In [72]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="safety",
    query_properties=["question"],
    return_properties=["question", "answer"],
    return_metadata=wvc.MetadataQuery(score=True),
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    print(o.metadata.score)

{
  "answer": "OSHA (Occupational Safety and Health Administration)",
  "question": "The government admin. was created in 1971 to ensure occupational health & safety standards"
}
3.333716630935669
{
  "answer": "France",
  "question": "Royale, Joseph, and Devil's Islands make up the Safety Islands owned by this country"
}
3.237332820892334
{
  "answer": "Devil's Island",
  "question": "The Safety Islands off French Guiana consist of Royale, Saint-Joseph & this diabolical island"
}
3.237332820892334


In [74]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="food",
    query_properties=["question^2", "answer"],
    return_properties=["question", "answer"],
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "cake",
  "question": "Devil's food & angel food are types of this dessert"
}
{
  "answer": "a closer grocer",
  "question": "A nearer food merchant"
}
{
  "answer": "food stores (supermarkets)",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}


In [76]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="food wine", # search for food or wine
    query_properties=["question"],
    return_properties=["question"], # only return question property
    limit=5
)

for o in response.objects:
    print(o.properties["question"])

Wine, a ship, Croce's time
Devil's food & angel food are types of this dessert
Type of event in Cana at which Jesus turned water into wine
A nearer food merchant
Sparkling wine sold under the name Champagne must come from this region in Northeast France


In [78]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.bm25(
    query="food",
    filters=wvc.Filter("round").equal("Double Jeopardy!"),
    return_properties=["answer", "question", "round"], # return these properties
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "food stores (supermarkets)",
  "round": "Double Jeopardy!",
  "question": "This type of retail store sells more shampoo & makeup than any other"
}
{
  "answer": "honey",
  "round": "Double Jeopardy!",
  "question": "The primary source of this food is the Apis mellifera"
}
{
  "answer": "pseudopods",
  "round": "Double Jeopardy!",
  "question": "Amoebas use temporary extensions called these to move or to surround & engulf food"
}


In [80]:
prompt = "Convert the following into a question for twitter. Include emojis for fun, but do not include the answer: {question}."

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    query="World history",
    limit=2,
    return_properties=["question", "answer"],
    single_prompt=prompt
)

# print source questions and generated responses
for o in response.objects:
    print(o.properties["question"])
    print(o.generated)

Including, in 19th century, one quarter of world's land & people, the sun never set on it
🌍 Did you know that in the 19th century, one quarter of the world's land and people were under its rule? ☀️ Can you guess which empire never saw the sun set on its territories? #TriviaTime #HistoryFun
From Menes to the Ptolemys, this country had more kings than any other in ancient history
🤔 Which country in ancient history had more kings than any other, from Menes to the Ptolemys? 👑🏛️ #HistoryTrivia


In [82]:
prompt = "Convert the following into a question for twitter. Include emojis for fun, but do not include the answer: {question}."

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    return_properties=["question", "answer"],
    query="World history",
    limit=2,
    single_prompt=prompt
)

# print source questions and generated responses
for o in response.objects:
    print(o.properties["question"])
    print(o.generated)

Including, in 19th century, one quarter of world's land & people, the sun never set on it
🌍 Did you know that in the 19th century, one quarter of the world's land and people belonged to a certain empire? ☀️ Can you guess which empire it was? #TriviaTime #HistoryFun
From Menes to the Ptolemys, this country had more kings than any other in ancient history
🤔 Which country in ancient history had more kings than any other, from Menes to the Ptolemys? 👑🏛️ #AncientHistoryTrivia


In [84]:
prompt = "Convert this quiz question: {question} and answer: {answer} into a trivia tweet."

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    return_properties=["question", "answer"],
    query="World history",
    limit=2,
    single_prompt=prompt
)

# print source properties and generated responses
for o in response.objects:
    print(o.properties)
    print(o.generated)

{'answer': 'the British Empire', 'question': "Including, in 19th century, one quarter of world's land & people, the sun never set on it"}
🌍 Did you know that in the 19th century, the British Empire covered a staggering one-quarter of the world's land and population? ☀️ With its vast reach, the sun never set on this mighty empire! Can you guess the answer? #TriviaTuesday #BritishEmpire
{'answer': 'Egypt', 'question': 'From Menes to the Ptolemys, this country had more kings than any other in ancient history'}
🔮 Trivia Time! 🔍 Which country in ancient history had more kings than any other? 🤔

A) Greece
B) Rome
C) Egypt
D) Persia

Think you know the answer? Reply with your guess! 🗒️✨ #TriviaTuesday #AncientHistory


In [86]:
task = "What do these animals have in common, if anything?"

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    return_properties=["question", "answer"],
    query="Cute animals",
    limit=3,
    grouped_task=task
)

# print the generated response
print(response.generated)

The commonality among these animals is that they have all been featured as answers or questions on the game show Jeopardy!.


In [93]:
task = "What do these animals have in common, if anything?"

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.generate.near_text(
    return_properties=["question", "answer"],
    query="Australian animals",
    limit=3,
    grouped_task=task,
    grouped_properties=["answer", "question"]
)

# print source properties
for o in response.objects:
    print(o.properties)

# print the generated response
print(response.generated)

{'answer': 'sheep', 'question': 'Australians call this animal a jumbuck or a monkey'}
{'answer': 'Australia', 'question': 'An island named for the animal seen <a href="http://www.j-archive.com/media/2000-03-10_J_01.jpg" target="_blank">here</a> belongs to this country [kangaroo]'}
{'answer': 'Kangaroo', 'question': 'Found chiefly in Australia, the wallaby is a smaller type of this marsupial'}
The commonality between these animals is that they are all found in Australia.


In [127]:
# Check that latest client is still borked. it is.

!pip install --pre -U "weaviate-client==4.*"

  Using cached weaviate_client-4.4b0-py3-none-any.whl.metadata (3.3 kB)
Using cached weaviate_client-4.4b0-py3-none-any.whl (220 kB)
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.3b1
    Uninstalling weaviate-client-4.3b1:
      Successfully uninstalled weaviate-client-4.3b1


In [95]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    return_properties=["question", "answer"],
    query="flying",
    limit=10,
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "on their stomachs",
  "question": "W. & O. Wright felt passengers wouldn't mind flying in this position they 1st flew in themselves"
}
{
  "answer": "flying the mail",
  "question": "In 1926 Lindbergh had to parachute out of planes 4 times while employed to do this"
}
{
  "answer": "a hot air balloon",
  "question": "In 1783 Benjamin Franklin saw the first piloted flight of this type of transport while in Paris"
}
{
  "answer": "a falcon",
  "question": "The fastest flying animal is the peregrine species of this bird of prey"
}
{
  "answer": "Pterodactyl",
  "question": "The name of this prehistoric reptile, the largest known flying animal, means \"wing finger\""
}
{
  "answer": "hot air balloons",
  "question": "These in the skies of Albuquerque on October 3, 1999 were a fine example of Charles' Law in action"
}
{
  "answer": "Lizards",
  "question": "In the East Indies certain species of this reptile are called flying dragons because they can glide from tree to tree"
}

In [96]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.over_all(total_count=True)

print(response.total_count)

10000


In [97]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.over_all(
    return_metrics=wvc.Metrics("answer").text(
        top_occurrences_count=True,
        top_occurrences_value=True
    )
)

# get the top_occurrences object
top_occurrences = response.properties["answer"].top_occurrences

# print top occurrences
for item in top_occurrences:
    print(item)

TopOccurrence(occurs=19, value='Australia')
TopOccurrence(occurs=18, value='Hawaii')
TopOccurrence(occurs=16, value='Boston')
TopOccurrence(occurs=15, value='French')
TopOccurrence(occurs=15, value='India')


In [99]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.over_all(
    return_metrics=wvc.Metrics("points").number(sum_=True, maximum=True, minimum=True),
)

print(response.properties["points"].sum_)
print(response.properties["points"].maximum)

6324100
10000


In [100]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate_group_by.over_all(
    group_by="round"
)

# print rounds names and the count for each
for group in response:
    print(f"Value: {group.grouped_by.value} Count: {group.total_count}")

Value: Double Jeopardy! Count: 5193
Value: Jeopardy! Count: 4522
Value: Final Jeopardy! Count: 285


In [101]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.near_text(
    query="animals in space",
    return_metrics=wvc.Metrics("points").number(sum_=True),
    object_limit=10,
)

print (response.properties["points"].sum_)

4600


In [102]:
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.near_text(
    query="animals in space",
    distance=0.19,
    object_limit=10,
    return_metrics=wvc.Metrics("points").number(sum_=True),
)

print (response.properties["points"].sum_)

3000


In [103]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.over_all(
    filters=Filter("round").equal("Final Jeopardy!")
)

print (response.total_count)

285


In [105]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    filters=Filter("round").equal("Double Jeopardy!"),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "Antimatter",
  "question": "In 1995 scientists in Switzerland created atoms of this substance that may not exist naturally in our universe"
}
{
  "answer": "The Manhattan Project",
  "question": "This \"project\" to develop a fission bomb was established in August 1942"
}
{
  "answer": "garage",
  "question": "This French word originally meant \"a place where one docks\" a boat, not a car"
}


In [107]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.near_text(
    return_properties=["question", "answer"],
    query="fashion icons",
    filters=Filter("points").greater_than(200),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "fashion designers",
  "question": "Ted Lapidus, Guy Laroche, Christian Lacroix"
}
{
  "answer": "Dapper Flapper",
  "question": "A stylish young woman of the 1920s"
}
{
  "answer": "Women's Wear Daily",
  "question": "This daily chronicler of the fashion industry launched \"W\", a bi-weekly, in 1972"
}


In [109]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    filters=Filter("answer").like("*inter*"),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "interglacial",
  "question": "This term refers to the warm periods within ice ages; we're in one of those periods now"
}
{
  "answer": "the Interior",
  "question": "In 1849, Thomas Ewing, \"The Logician of the West\", became the USA's first Secy. of this Cabinet Dept."
}
{
  "answer": "Interlaken, Switzerland",
  "question": "You can view the Jungfrau Peak from the main street of this town between the Brienz & Thun Lakes"
}


In [111]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    filters=Filter("round").equal("Double Jeopardy!") &
            Filter("points").less_than(600),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "The Manhattan Project",
  "question": "This \"project\" to develop a fission bomb was established in August 1942"
}
{
  "answer": "Amy Carter",
  "question": "On September 1, 1996 this former first daughter married Jim Wentzel at the Pond House near Plains"
}
{
  "answer": "Franklin D. Roosevelt",
  "question": "8 days after his March 4, 1933 inauguration, he gave his first Fireside Chat"
}


In [112]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_properties=["question", "answer"],
    filters=Filter("question").like("*nest*") &
            (Filter("points").greater_than(700) | Filter("points").less_than(300)),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

{
  "answer": "Carpathians",
  "question": "The Dnestr & Vistula rivers rise in these mountains"
}
{
  "answer": "The Mambo Kings",
  "question": "In a Pulitzer Prize-winning novel by Oscar Hijuelos, Cesar & Nestor Castillo lead this dance band"
}
{
  "answer": "Owl",
  "question": "The screech type of this creature may make its nest in a Saguaro cactus"
}


In [5]:
from weaviate.classes import Filter

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    #return_properties=["question", "answer"],
    filters=Filter(["hasCategory", "JeopardyCategory", "title"]).like("*Sport*"),
    limit=3
)

# print result objects 
for o in response.objects:
    print(json.dumps(o.properties, indent=2))
    

{
  "answer": "Sampan",
  "question": "Smaller than a junk, this Oriental boat usually has a cabin with a roof made of mats"
}
{
  "answer": "the bicycle",
  "question": "The Michauxes added pedals to the front wheel of a velocipede & became the \"fathers of\" this vehicle"
}
{
  "answer": "hot air balloons",
  "question": "During the Cold War, 2 different families escaped over the Berlin Wall using these lighter-than-air vehicles"
}


In [9]:
jeopardy.config.get()
response

_QueryReturn(objects=[_Object(uuid=UUID('dfd7f9a2-509b-5498-b473-690ac31fd4ec'), metadata=None, properties={'answer': 'Sampan', 'question': 'Smaller than a junk, this Oriental boat usually has a cabin with a roof made of mats'}, vector=None), _Object(uuid=UUID('94438094-4f07-58e7-84c3-6662eae0fa34'), metadata=None, properties={'answer': 'the bicycle', 'question': 'The Michauxes added pedals to the front wheel of a velocipede & became the "fathers of" this vehicle'}, vector=None), _Object(uuid=UUID('53922edd-b280-5c1d-aee6-837fdb080bda'), metadata=None, properties={'answer': 'hot air balloons', 'question': 'During the Cold War, 2 different families escaped over the Berlin Wall using these lighter-than-air vehicles'}, vector=None)])

In [3]:
print(weaviate.__version__)

4.4b0


In [10]:
print("or something")

or something
